# Generación de intervalo de confianza

In [6]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Nota: Para este ejemplo asumiremos que los datos de entrenamiento son iguales a los datos de prueba

In [8]:
Y= np.array([[3],[1],[8],[3],[5]])
Y

array([[3],
       [1],
       [8],
       [3],
       [5]])

In [14]:
X= np.array([[1,3],[1,1],[1,5],[1,2],[1,4]])
X

array([[1, 3],
       [1, 1],
       [1, 5],
       [1, 2],
       [1, 4]])

In [16]:
XT_X= np.matmul(np.matrix.transpose(X),X)
XT_X

array([[ 5, 15],
       [15, 55]])

In [18]:
XT_X_inv= np.linalg.inv(XT_X)
XT_X_inv

array([[ 1.1, -0.3],
       [-0.3,  0.1]])

In [20]:
XT_Y= np.matmul(np.matrix.transpose(X),Y)
XT_Y

array([[20],
       [76]])

In [24]:
betas= np.matmul(XT_X_inv,XT_Y)
betas

array([[-0.8],
       [ 1.6]])

In [28]:
# Cálculo de Y s Pronosticadas
Y_Pred= np.matmul(X,betas)
Y_Pred

array([[4. ],
       [0.8],
       [7.2],
       [2.4],
       [5.6]])

In [30]:
# Cálculo de residuales
Resid= Y-Y_Pred
Resid

array([[-1. ],
       [ 0.2],
       [ 0.8],
       [ 0.6],
       [-0.6]])

In [32]:
# Cálculo de sumas de residuales al cuadrado
RSS= np.matmul(np.matrix.transpose(Resid), Resid)
RSS

array([[2.4]])

In [34]:
#Cálculo de la suma total de cuadrados
TSS= np.matmul(np.matrix.transpose(Y),Y)-len(Y)*(Y.mean()**2)
TSS

array([[28.]])

In [36]:
# Cálculo del coefiiente de determinación
R_cuadr= float(1-(RSS/TSS))
R_cuadr

0.9142857142857144

In [40]:
# Cálculo del coefiiente de determinación ajustada
R2_ajus= float(1-((RSS/(X.shape[0]-X.shape[1]))/(TSS/(X.shape[0]-1))))
R2_ajus

0.8857142857142858

In [48]:
# Varianza del error
s_cuad= RSS/(len(Y)-X.shape[1])
s_cuad

array([[0.8]])

In [50]:
# Desviación
import math
s=math.sqrt(s_cuad)
s

0.8944271909999157

In [52]:
# Obtención del valor crítico de la t de student
import scipy.stats

# Grado de libertad (número de observaciones - némero de variables +1) n-(k+1)
grados_libertad= len(Y)-X.shape[1]
Confianza= .95
q= (1-Confianza)/2
t_critico= abs(scipy.stats.t.ppf(q, df=grados_libertad))
t_critico

3.182446305284263

In [54]:
# Vector de valores particulares de X (1 es por el intercepto y 7 es el valor que tendría X para pronosticaqr Y
f=np.array([[1],[7]])
f

array([[1],
       [7]])

In [58]:
Margen_error= t_critico*(s*(float(np.matmul(np.matmul(np.matrix.transpose(f), XT_X_inv),f))**.5))
Margen_error

3.818935566341115

In [62]:
# Pronostico Y con X =7
Pron_puntual= float(np.matmul(np.matrix.transpose(f),betas))
Pron_puntual

10.400000000000006

In [64]:
# Límites del  intervalo de confianza
# cuando x=7 Y va a estar en un intervalo ente 6.58 y 14.21 con un 95% de confianza

lim_inf= Pron_puntual-Margen_error
lim_sup= Pron_puntual+Margen_error
print("intervalos de confianza:,(",lim_inf,",",lim_sup, ")")

intervalos de confianza:,( 6.581064433658891 , 14.21893556634112 )


# Validación de supuestos de regresión

# Supuesto 1 Jarque Bera

In [69]:
import scipy
Resid

array([[-1. ],
       [ 0.2],
       [ 0.8],
       [ 0.6],
       [-0.6]])

In [71]:
# Cálculo de la asimetría de los residuales
skew=float(scipy.stats.skew(Resid,bias=True))
skew

-0.2886751345948135

In [75]:
# Cálculo de la curtosis de residuales (se le pone false para que funcione con el estadístico Jarque Bera)
Kurtosis= float(scipy.stats.kurtosis(Resid,fisher=False))
Kurtosis

1.4499999999999993

In [77]:
# Estadístico Jarque Bera

Jarque_bera= (len(Y)/6*(skew**2+(Kurtosis-3)**2/4))
Jarque_bera
              

0.5699652777777785

In [79]:
Nivel_confianza=.95
scipy.stats.chi2.ppf(Nivel_confianza, df=2)

5.991464547107979

### Conclusión: dado que el estadistico Jarque Bera no es mayor al nivel crítico 5.99, no podemos rechazar la hipótesis de existencia de normalidad en los residuales

# Supuesto 2: Inexistencia de autocorrelación entre residuales

In [90]:
from statsmodels.formula.api import ols

In [98]:
# Sin intercepto la X
y_df= pd.DataFrame(Y)
x_df=pd.DataFrame(X[:,1:2])
x_df

,0
0,3
1,1
2,5
3,2
4,4


In [106]:
y_df

,0
0,3
1,1
2,8
3,3
4,5


In [104]:
df= pd.concat([y_df,x_df.reindex(y_df.index)],axis=1)
df.columns=["Y","X1"]
df

,Y,X1
0,3,3
1,1,1
2,8,5
3,3,2
4,5,4


In [120]:
#Ajuste de regresión lineal múltiple (sí hay más variable independientes se pone X1+X2+X3....ETC
model=ols('Y ~ X1',data=df).fit()

from statsmodels.stats.stattools import durbin_watson

#Prueba de durbin_watson
durbin_watson(model.resid)

1.3666666666666656

### Conclusión: dado que durbin watson no es aprox. igual a 2, podemos pensar que existe autocorrelación entre los residuales
### no es bueno, ya que los intervalos de confianza no son tan confiables

# Supuesto 3: Homocedasticidad (igualdad de varianzas)
# también afecta las confiabilidad de los intervalos de confianza

In [130]:
ResidCuac=Resid**2
ResidCuac=pd.DataFrame(ResidCuac)
ResidCuac

,0
0,1.00
1,0.04
2,0.64
3,0.36
4,0.36


In [146]:
X1=df.iloc[:,1]
X1_df=pd.DataFrame(X1)

X1Cuad= X1**2
X1Cuad_df=pd.DataFrame(X1Cuad)

In [148]:
df_Aux=pd.concat([ResidCuac,X1_df.reindex(y_df.index),X1Cuad_df.reindex(y_df.index)],axis=1)
df_Aux.columns=["Residual","X1","X1Cuad"]
df_Aux

,Residual,X1,X1Cuad
0,1.00,3,9
1,0.04,1,1
2,0.64,5,25
3,0.36,2,4
4,0.36,4,16


In [150]:
# Ajuste de regresión lineal múltiple
modelAux= ols("Residual ~ X1 + X1Cuad", data=df_Aux).fit()
RsqAux=modelAux.rsquared
RsqAux

0.5326278659612038

In [152]:
Estadistico=len(Y)*RsqAux
Estadistico

2.663139329806019

In [154]:
Nivel_confianza=.95
scipy.stats.chi2.ppf(Nivel_confianza, df=2)

5.991464547107979

### Conclusión: al no superar el valor crítico (5.99) nuestro estadístico (2.66), no hay evidencia de Heterocedasticidad (desigualdad de varianza de los residuales)

In [160]:
# Alternativa para prueba White con Python (al ser el valor de p mayor al 5%, no se puede rechazar la hipótesis nula de que hay homocedastiidad

from statsmodels.stats.diagnostic import het_white
white_test=het_white(model.resid,model.model.exog)
print("Estadístico de White", white_test[0])
print("Valor p", white_test[1])

Estadístico de White 2.66313932980599
Valor p 0.26406244627058784


### Conclusión: a un nivel de alpha de 5%  como tenemos un valor de p superior a alpha, no podemos rechazar la hipótesis de homocedasticidad (lo cual implica que no existe evidencia de lo contrario (heterocedasticidad)

# Supuesto 4: inexistencia de Multicolinealidad (sólo se puede a partir de 2 variables

### en este caso no aplica realizarla, ya que sólo tenemos una variable regresora (X1). En modelos con más variables independientes sí habría que realizarla